# Continuous Optimisation HW1

In [2]:
# Imports
import numpy as np
import scipy

In [4]:
# Question 1

def h(x: np.array):
    return np.e**(-np.inner(x, x)/0.01)  # Always take sigma = 0.1


# def phi(x, P):


<class 'numpy.ndarray'>
